# NanoPyx Methods

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import stackview

### Channel Registration

In [ ]:
from nanopyx.methods import channel_registration

# Generating example data 
from nanopyx.core.generate.beads import generate_channel_misalignment

input_img = generate_channel_misalignment()
stackview.slice(input_img, continuous_update=True)

In [ ]:
aligned_img = channel_registration.estimate_channel_registration(input_img,0, 200, 3, 0.5, save_ccms=False, ccms_save_path="",save_translation_masks=False, translation_mask_save_path="", apply=True)
stackview.slice(aligned_img, continuous_update=True)

### Drift Correction

In [ ]:
from nanopyx.methods import drift_alignment

# Generating example data
from nanopyx.core.generate.beads import generate_timelapse_drift

input_img=generate_timelapse_drift(n_objects=5, shape=(50, 500, 500), drift=1)
stackview.slice(input_img,continuous_update=True)

In [ ]:
aligned_img = drift_alignment.estimate_drift_alignment(input_img, save_drift_table_path="",ref_option=0,apply=True)
stackview.slice(aligned_img, continuous_update=True)

### SRRF

In [ ]:
from nanopyx.methods import SRRF

# Generating example data
from nanopyx.data.download import ExampleDataManager
downloader = ExampleDataManager()
input_img = downloader.get_ZipTiffIterator(
    "SMLMS2013_HDTubulinAlexa647", as_ndarray=True).astype(np.float32)
stackview.slice(input_img, continuous_update=True)

In [ ]:
srrfed_img = SRRF(input_img,magnification=1)
plt.imshow(np.mean(srrfed_img[0],axis=0))
plt.show()

### eSRRF

In [ ]:
from nanopyx.methods import eSRRF

# Generating example data
from nanopyx.data.download import ExampleDataManager
downloader = ExampleDataManager()
input_img = downloader.get_ZipTiffIterator(
    "SMLMS2013_HDTubulinAlexa647", as_ndarray=True).astype(np.float32)
stackview.slice(input_img, continuous_update=True)

In [ ]:
esrrfed_img = eSRRF(input_img,magnification=1)
plt.imshow(np.mean(esrrfed_img[0],axis=0))
plt.show()

### Fourier ring correlation

In [ ]:
from nanopyx.core.analysis.frc import FIRECalculator

# Generating example data
from nanopyx.data.download import ExampleDataManager
downloader = ExampleDataManager()
input_img = downloader.get_ZipTiffIterator(
    "SMLMS2013_HDTubulinAlexa647", as_ndarray=True).astype(np.float32)
stackview.slice(input_img, continuous_update=True)

In [ ]:
calculator = FIRECalculator(pixel_size=1, units="px")
fire = calculator.calculate_fire_number(input_img[0], input_img[50])
plot = calculator.plot_frc_curve()
plt.imshow(plot)
plt.axis('off')
plt.show()

### Decorrelation analysis

In [ ]:
from nanopyx.core.analysis.decorr import DecorrAnalysis

# Generating example data
from nanopyx.data.download import ExampleDataManager
downloader = ExampleDataManager()
input_img = downloader.get_ZipTiffIterator(
    "SMLMS2013_HDTubulinAlexa647", as_ndarray=True).astype(np.float32)
stackview.slice(input_img, continuous_update=True)

In [ ]:
decorr = DecorrAnalysis(pixel_size=1, units="pixel", do_plot=True)
decorr.run_analysis(input_img[0])
plot = decorr.plot_results()

plt.imshow(plot)
plt.axis('off')
plt.show()

### Non-local means denoising

In [ ]:
from nanopyx.core.transform import NLMDenoising
from nanopyx.core.generate.noise_add_mixed_noise import add_mixed_gaussian_poisson_noise
# Generating example data

input_img = np.mean(esrrfed_img[0],axis=0)
#input_img = (input_img-input_img.min())/(input_img.max()-input_img.min())

add_mixed_gaussian_poisson_noise(input_img, gauss_sigma=20, gauss_mean=1)

plt.imshow(input_img)

In [ ]:
denoiser = NLMDenoising()
denoised_img = denoiser.run(input_img,patch_size=5,patch_distance=100,sigma=20,h=15)

plt.imshow(denoised_img)